# MNIST Dataset Example

This Jupyter Notebook demonstrates how to build and use a neural network with convolutional and pooling layers to process and classify images from the MNIST dataset.

## Download the Dataset

To begin, you need to download the `mnist.npz` dataset file into your working directory. This file contains pre-split training and testing sets of the MNIST dataset. You can download it from the following link:

[Download MNIST Dataset](https://s3.amazonaws.com/img-datasets/mnist.npz)

Ensure that the downloaded file is placed in the same directory as this notebook to load the dataset seamlessly.

In [ ]:
import numpy as np

from src import BaseModel

from src.loss import CategoricalCrossentropy
from src.optimizer import Adam
from src.tensor import Tensor, op
from src.structure import Dense, Convolution, Dropout, MaxPool, Flatten, LeakyRelu, Softmax
from src.encode import OneHotEncoder
from src.preprocessing import DataLoader, min_max_scaler

In [ ]:
path = './mnist.npz'

with np.load(path, allow_pickle=True) as f:
    x_train, y_train = f["x_train"], f["y_train"]
    x_test, y_test = f["x_test"], f["y_test"]

x_train = min_max_scaler(np.array(x_train), 0, 1)
x_test = min_max_scaler(np.array(x_test), 0, 1)

x_train = op.expand_dims(x_train, axis=1)
x_test = op.expand_dims(x_test, axis=1)

classes = tuple(np.unique(y_train))

In [ ]:
class Model(BaseModel):
    def __init__(self) -> None:
        self.layers = [
            Convolution(channels=8, kernel_shape=(3, 3), padding=1),
            LeakyRelu(),
            MaxPool(channels=8, filter_shape=(2, 2)),
            Convolution(channels=16, kernel_shape=(3, 3), padding=1),
            LeakyRelu(),
            MaxPool(channels=16, filter_shape=(2, 2)),
            Convolution(channels=32, kernel_shape=(3, 3), padding=1),
            LeakyRelu(),
            Flatten(),
            Dense(64),
            LeakyRelu(),
            Dropout(p=0.1),
            Dense(10),
            Softmax(),
        ]

        self.encoder = OneHotEncoder(classes)

    def __call__(self, inputs: Tensor) -> Tensor:
        return inputs.sequential(self.layers)

    def train(self, steps: int) -> None:
        data = DataLoader(x_train, y_train, batch_size=32)

        opt = Adam(self.parameters(), 1e-6)

        loss_func = CategoricalCrossentropy()

        for step in range(steps):
            x, y = next(data)

            opt.zero_grad()

            loss = loss_func(self(x), self.encoder(y))
            loss.backward()

            opt.step()

            if step % 1000 == 0:
                print(f"loss: {loss.mean()}")

        print(f"train: {self.evaluate(x_train, y_train)}")

    @Tensor.no_grad()
    def evaluate(self, data: Tensor, expected: Tensor) -> float:
        correct = 0
        total = 0

        for i in range(len(data)):
            total += 1

            if self.encoder.decode(self(data[i])) == expected[i].item():
                correct += 1

        return correct / total

nn = Model()

nn.train(100_000)

print(f"test: {nn.evaluate(x_test, y_test)}, train: {nn.evaluate(x_train, y_train)}")
